<h1>Clean Up<span class="tocSkip"></span></h1>

In this jupyter I am going to clean up a DataFrame that allows us to make the visualization and machine learning model in order to see what is the best fuel.

This model is made thanks to a database downloaded from [Kaggle](https://www.kaggle.com/anderas/car-consume?select=measurements.csv) in which there are more than 380 car values information. data is based of driving always the same car and taking almost always the same route.


# Import libraries

In [1]:
import src.limpieza as lm
import pandas as pd
import numpy as np
import re

# Import DataFrame

## Download data from [Kaggle](https://www.kaggle.com/anderas/car-consume?select=measurements.csv)

In [2]:
#lm.download_kaggle()

## Open `.csv` file

In [3]:
data =pd.read_csv("data/measurements.csv")

In [4]:
data.head()

,distance,consume,speed,temp_inside,temp_outside,specials,gas_type,AC,rain,sun,refill liters,refill gas
0,28,5,26,"21,5",12,NaN,E10,0,0,0,45,E10
1,12,"4,2",30,"21,5",13,NaN,E10,0,0,0,NaN,NaN
2,"11,2","5,5",38,"21,5",15,NaN,E10,0,0,0,NaN,NaN
3,"12,9","3,9",36,"21,5",14,NaN,E10,0,0,0,NaN,NaN
4,"18,5","4,5",46,"21,5",15,NaN,E10,0,0,0,NaN,NaN


# Preliminary Exploration 

## Dimension

In [5]:
data.shape

(388, 12)

## Data Types


In [6]:
data.dtypes

distance         object
consume          object
speed             int64
temp_inside      object
temp_outside      int64
specials         object
gas_type         object
AC                int64
rain              int64
sun               int64
refill liters    object
refill gas       object
dtype: object

### Objet to integer
As we can see the first two columns are objects although in reality the value entered is an integer, so we are going to change the object type for these columns.

In [7]:
data["distance"].loc[2]

'11,2'

In [8]:
data["temp_inside"].loc[2].replace(",",".")

'21.5'

In [9]:
type(data["temp_inside"].loc[232])

str

As we can see the data is a string and separated by a comma instead of a dot, so this could mean an error when we try to pass it to float, so the first thing we have to do is to change that `","` for a `"."`.

In [10]:
data["distance"] = data["distance"].apply(lm.dot_comma)

In [11]:
data["distance"] = data["distance"].astype(float)

Now we have to do the same but with the other rows, so the best solution is to create a function that do it.

In [12]:
data["consume"] = lm.str_to_float(data["consume"])

In [13]:
data["temp_inside"] = lm.str_to_float(data["temp_inside"])

In [14]:
data["refill liters"] = lm.str_to_float(data["refill liters"])

In [15]:
data.dtypes

distance         float64
consume          float64
speed              int64
temp_inside      float64
temp_outside       int64
specials          object
gas_type          object
AC                 int64
rain               int64
sun                int64
refill liters    float64
refill gas        object
dtype: object

## Null values

Let´s check it out how many null values are there in each column in order to see wich columns are useful.

In [16]:
data.isnull().sum()

distance           0
consume            0
speed              0
temp_inside       12
temp_outside       0
specials         295
gas_type           0
AC                 0
rain               0
sun                0
refill liters    375
refill gas       375
dtype: int64

In [17]:
data.rain.value_counts()

0    340
1     48
Name: rain, dtype: int64

### Let´s check the columns with more null values

#### Specials
This column has 295 null values (`76,03%`) and in it there is meteorological information, although this information is available in the following columns.

In [18]:
data.specials.value_counts()

rain                  32
sun                   27
AC rain                9
ac                     8
AC                     6
sun ac                 3
snow                   3
AC Sun                 1
AC snow                1
half rain half sun     1
ac rain                1
AC sun                 1
Name: specials, dtype: int64

In [19]:
data["refill liters"].value_counts()

45.0    2
39.0    2
37.7    2
10.0    1
38.0    1
38.3    1
37.2    1
41.0    1
37.6    1
37.0    1
Name: refill liters, dtype: int64

In [20]:
data["refill gas"].value_counts()

SP98    8
E10     5
Name: refill gas, dtype: int64

###  Fixing null values
Now I am going to fill in the unknown values of useful $m^2$, because there are `13.514` null values for this column but it is a value that we can 'predict'. To do this I am going to find out what average percentage of $m^2$ is the usable area with respect to the constructed area.

This factor should vary depending on the thickness of the walls, the existence of terraces or partitions. It is up to the technician to define this value, which is usually between 0.90 and 0.80. So a correct value would be in this range.

`I do this process even though I know that I might delete the useful square metres column at a later stage due to the high correlation with the built square metres column.`

## Creating new columns

In this step I am going to create different columns that I believe will be useful later on for the machine learning model.

## Fuel price
What I am going to do now is to create a new column in which I enter the unit price of fuel for each type.

In [21]:
fuel_price = {"E10" : 1.38,
              "SP98" : 1.46}

In [22]:
data["Fuel_Price"] = data.gas_type.map(fuel_price)

##  Price of the journey
What I am going to do now is to create a new column in which I enter price for each journey.

In [32]:
data["Journey_Price"] = round(data["distance"] * (data["consume"]/100) * data["Fuel_Price"],2)

In [33]:
data.sample(5)

,distance,consume,speed,temp_inside,temp_outside,gas_type,AC,rain,sun,Fuel_Price,Journey_Price
91,12.2,5.8,60,20.0,11,SP98,0,0,0,1.46,1.03
266,16.0,4.0,40,22.0,10,E10,0,0,0,1.38,0.88
309,31.9,4.3,33,22.0,16,SP98,0,0,0,1.46,2.00
79,34.8,4.0,28,20.0,4,SP98,0,0,0,1.46,2.03
308,10.1,4.2,35,22.0,16,SP98,0,0,0,1.46,0.62


# Delete values

## Drop unuseful columns

In [29]:
data.drop(["refill liters", "refill gas", "specials"],axis=1, inplace=True)

## NaN values

In [30]:
data.dropna(axis=0, how="any", inplace=True)

# Last Check

In [35]:
data.sample(5)

,distance,consume,speed,temp_inside,temp_outside,gas_type,AC,rain,sun,Fuel_Price,Journey_Price
208,18.1,3.6,36,20.0,19,SP98,0,0,0,1.46,0.95
381,5.5,3.7,33,24.5,28,SP98,0,0,1,1.46,0.30
13,24.7,5.1,58,21.5,12,E10,0,0,0,1.38,1.74
44,5.4,3.3,32,21.5,7,SP98,0,0,0,1.46,0.26
89,13.9,5.6,22,20.0,8,SP98,0,0,0,1.46,1.14


# Export DataFrame

In [36]:
data.to_csv("data/cars_limpio.csv")